In [1]:
from collections import defaultdict
import nltk
import os
import numpy as np
from nltk.corpus import machado, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import string
from collections import defaultdict
from nltk.stem.snowball import PortugueseStemmer
import enchant
from gensim import corpora

In [2]:
textos = []
for p, d, f in os.walk(r'machado/machado'):
    #print( p,d,f)
    if f:
        for fileid  in f:
            if not fileid.endswith('.txt'):
                continue
            with open(os.path.join(p,fileid), encoding='iso-8859-1') as g:
                textos.append(g.read())

In [3]:
swu = stopwords.words('portuguese') + list (string.punctuation)
stemmer = PortugueseStemmer()

In [4]:
#textos = textos[:4]

In [5]:
def clean_sentence(texto : str):
    return [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
textos = [*map(clean_sentence,textos)]

In [6]:
all_words = set()
for text in textos:
    all_words |= set(text)
ordered_words = list(sorted(all_words))
index_word = dict([(word,i) for i, word in enumerate(ordered_words)])

In [8]:
matrix = [[0]*len(textos) for _ in range(len(all_words))]
for i, document in enumerate(textos):
    for word in document:
        j = index_word[word]
        matrix[j][i] += 1
matrix = np.matrix(matrix)
binary_matrix = (matrix >= 1)*1

## Questão 1

In [9]:
def frequency_from(word, matrix):
    clean_word = clean_sentence(word)[0]
    if clean_word not in index_word:
        return [[0]*matrix.shape[1]]
    word_index = index_word[clean_word]
    
    line = matrix[word_index]
    return line/np.sum(line)

def binary_frequency(word):
    return frequency_from(word,binary_matrix)

def word_frequency(word):
    return frequency_from(word,matrix)

In [10]:
binary_frequency('agora')

matrix([[0.00487805, 0.        , 0.        , 0.00487805, 0.00487805,
         0.        , 0.        , 0.        , 0.00487805, 0.        ,
         0.00487805, 0.00487805, 0.        , 0.00487805, 0.        ,
         0.        , 0.00487805, 0.00487805, 0.        , 0.00487805,
         0.00487805, 0.00487805, 0.00487805, 0.00487805, 0.        ,
         0.00487805, 0.00487805, 0.        , 0.        , 0.00487805,
         0.00487805, 0.00487805, 0.00487805, 0.        , 0.        ,
         0.00487805, 0.        , 0.        , 0.00487805, 0.        ,
         0.00487805, 0.        , 0.00487805, 0.00487805, 0.00487805,
         0.        , 0.00487805, 0.00487805, 0.00487805, 0.00487805,
         0.00487805, 0.00487805, 0.00487805, 0.00487805, 0.00487805,
         0.00487805, 0.00487805, 0.00487805, 0.00487805, 0.00487805,
         0.00487805, 0.00487805, 0.00487805, 0.00487805, 0.        ,
         0.        , 0.00487805, 0.00487805, 0.00487805, 0.00487805,
         0.00487805, 0.00487805, 0

In [11]:
word_frequency('agora')

matrix([[0.00070497, 0.        , 0.        , 0.00035249, 0.00140994,
         0.        , 0.        , 0.        , 0.00035249, 0.        ,
         0.00458231, 0.00105746, 0.        , 0.00035249, 0.        ,
         0.        , 0.00070497, 0.00140994, 0.        , 0.00105746,
         0.00140994, 0.00035249, 0.00035249, 0.00035249, 0.        ,
         0.00035249, 0.00070497, 0.        , 0.        , 0.00035249,
         0.00035249, 0.00387734, 0.00105746, 0.        , 0.        ,
         0.00845964, 0.        , 0.        , 0.00140994, 0.        ,
         0.00035249, 0.        , 0.00105746, 0.00176243, 0.00035249,
         0.        , 0.00035249, 0.0024674 , 0.00035249, 0.00105746,
         0.00070497, 0.00070497, 0.01762425, 0.0024674 , 0.00281988,
         0.0024674 , 0.00669722, 0.00352485, 0.00035249, 0.00281988,
         0.00070497, 0.00317237, 0.00176243, 0.00105746, 0.        ,
         0.        , 0.02220656, 0.00140994, 0.00387734, 0.00387734,
         0.00176243, 0.00035249, 0

In [12]:
def tfdf(word):
    clean_word = clean_sentence(word)[0]
    if clean_word not in index_word:
        return [[0]*matrix.shape[1]]
    word_index = index_word[clean_word]
    line = matrix[word_index]
    quantity_words_per_doc = matrix.sum(axis=0)
    tf = line/quantity_words_per_doc
    
    freq_word = ((tf > 0)*1).sum()
    assert freq_word > 0
    df = np.log(matrix.shape[1]/freq_word)
    assert df >= 0
    return tf*df

In [13]:
tfdf('agora')

matrix([[5.87186978e-04, 0.00000000e+00, 0.00000000e+00, 1.67729123e-04,
         1.03152224e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.14956845e-04, 0.00000000e+00, 2.77766347e-04, 5.72138777e-04,
         0.00000000e+00, 3.61749121e-04, 0.00000000e+00, 0.00000000e+00,
         8.53965137e-04, 2.14559055e-04, 0.00000000e+00, 1.26612192e-03,
         2.19202353e-04, 1.10097558e-04, 3.88745324e-04, 3.95491446e-04,
         0.00000000e+00, 3.61749121e-04, 3.92384713e-05, 0.00000000e+00,
         0.00000000e+00, 1.40571748e-04, 3.01357945e-04, 1.08771945e-04,
         1.85663500e-04, 0.00000000e+00, 0.00000000e+00, 2.90783982e-04,
         0.00000000e+00, 0.00000000e+00, 7.97906157e-04, 0.00000000e+00,
         1.64550142e-04, 0.00000000e+00, 2.18961037e-04, 2.73838325e-04,
         1.75140785e-04, 0.00000000e+00, 1.00231752e-04, 4.51930205e-04,
         3.05908652e-04, 1.35387295e-04, 3.93783060e-04, 1.28124776e-04,
         3.56194187e-04, 3.43724993e-04, 3.07326687

## Questão 2

In [14]:
def get_log_normalized(matrix):
    log_matrix = np.log(matrix+1)
    squared_log_matrix = np.square(log_matrix)
    sum_rows = squared_log_matrix.sum(axis=1)
    log_normalized_matrix = np.power(np.multiply(squared_log_matrix,np.power(sum_rows, -1)),0.5)
    return log_normalized_matrix

In [15]:
log_normalized_matrix = get_log_normalized(matrix)
log_normalized_matrix

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.07960309, 0.07960309,
         0.        ]])

## Questão 3

In [16]:
frequency = defaultdict(int)
for text in textos:
    for token in text:
        frequency[token] += 1
freq_texts = [[token for token in text if frequency[token] > 1]
         for text in textos]
dictionary = corpora.Dictionary(freq_texts)

In [17]:
def to_corpus(doc):
    new_vec = dictionary.doc2bow(doc.lower().split())
    corpus = [dictionary.doc2bow(text) for text in freq_texts]
    return corpus

In [18]:
[vet[:5] for vet in to_corpus('agora')[:5]]

[[(0, 4), (1, 1), (2, 1), (3, 2), (4, 2)],
 [(0, 5), (6, 1), (7, 5), (14, 4), (20, 1)],
 [(6, 1), (7, 1), (20, 1), (21, 1), (24, 1)],
 [(6, 1), (7, 18), (13, 1), (16, 1), (20, 1)],
 [(0, 3), (5, 1), (6, 1), (7, 1), (14, 2)]]